# LangGraph ChatBot 구축

#### add_message
**add_message**는 LangGraph에서 메시지 기반 애플리케이션을 구축할 때 사용하는 **특별한 reducer 함수**  

**주요기능**
- 두개의 메시지 리스트를 병합
- 동일한 ID를 가진 메시지가 있을 경우 새로운 메시지로 기존 메시지를 대체

**주요인자**
- left(messages) : 기본 메시지 리스트
- right(messages) : 병합할 리스트 또는 단일 메시지

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import add_messages

human_message = [HumanMessage(content="안녕하세요!", id="1")]
ai_message = [AIMessage(content="안녕하세요! 무엇을 도와드릴까요?", id="2")]

messages = add_messages(human_message, ai_message)
print(messages)

동일한 메시지 id가 있을 경우 가 있을 경우 대체됨

In [ ]:
human_message = [HumanMessage(content="야식 뭐먹을까요?", id="1")]

result = add_messages(messages, human_message)

print(result)

### add_messages의 핵심 구조

```python
from typing import Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
```

- "messages"키는 add_messages 리듀서라는 함수에 주석이 달려있어 
- LangGraph한테 기존 목록에 새로운 메시지를 추가하도록 지시하는 역할을 해줌

#### 구조 분석

1. **`messages`**: 메시지들을 저장할 필드
2. **`list[AnyMessage]`**: 모든 종류의 메시지를 담을 수 있는 리스트
3. **`Annotated[..., add_messages]`**: `add_messages` reducer를 사용하여 메시지 추가


> **Annotated**
> - Python의 typing 모듈에서 제공하는 타입 힌트, 기존 타입에 메타데이터를 추가가능
> - 코드 자체에 추가설명을 포함시켜 문서화하는 효과를 얻기 위해 사용
> - LangGraph에서는 Annotated를 사용하여 에이전트가 어떻게 합치고 저장해나갈지 선언적으로 정의할 수 있음.


#### 메시지 타입들

- **`HumanMessage`**: 사용자가 보낸 메시지
- **`AIMessage`**: AI가 생성한 응답 메시지
- **`SystemMessage`**: 시스템 설정 메시지
- **`ToolMessage`**: 도구 실행 결과 메시지


다음은 add_Messages를 활용해 ChatBot을 만드는 예제입니다.

**1. Messages State 정의하기**

ChatBot을 만들기 위해서는 먼저 **메시지를 저장할 수 있는 상태**를 정의해야 합니다.

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

# ChatBot을 위한 State 정의
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


add_messages 동작 확인해보기

실제로 add_messages가 어떻게 동작하는지 확인해보겠습니다.

**2. ChatBot Node 구현하기**

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

**ChatBot Node 구현**

ChatBot Node는 다음과 같은 역할을 수행합니다:
1. 현재 상태에서 **모든 메시지**를 가져오기
2. LLM에 메시지들을 전달하여 **응답 생성**
3. 생성된 응답을 **새로운 AIMessage**로 반환

In [ ]:
# ChatBot Node 함수 정의
def chatbot_node(state: State):
    
    # 1. 현재 상태에서 모든 메시지 가져오기
    messages = state["messages"]
    
    # 2. LLM 호출하여 응답 생성
    response = llm.invoke(messages)
    
    # 3. 생성된 응답을 새로운 상태로 반환
    # add_messages 덕분에 기존 메시지들이 보존되고 새 응답이 추가됨
    return {"messages": [response]}

**3. ChatBot 그래프 구축하기**

In [ ]:
from langgraph.graph import StateGraph, START, END

# ChatBot 그래프 구축
    
# 1. StateGraph 초기화
workflow = StateGraph(State)
    
# 2. ChatBot Node 추가
workflow.add_node("chatbot", chatbot_node)
    
# 3. 그래프 흐름 설정
workflow.add_edge(START, "chatbot")  # 시작 → chatbot
workflow.add_edge("chatbot", END)    # chatbot → 끝
    
# 4. 그래프 컴파일
graph = workflow.compile()

**4. 그래프 시각화**

In [ ]:
# 그래프 시각화 (선택사항)
from IPython.display import Image, display

try:
    # 그래프 구조를 Mermaid 다이어그램으로 표시
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    pass

**ChatBot 실행 및 테스트**

In [ ]:
# 첫 번째 대화
user_input = "안녕하세요! 자기소개를 해주세요."

result = graph.invoke({"messages": [user_input]})

In [ ]:
print(result['messages'][-1].content)

**스트리밍(streaming) 실행**  
→ 그래프내에서 각 노드가 실행되는 과정을 차례대로 받아올 수 있습니다.  
→ 각 노드별 처리과정 (입력 -> 실행 -> 출력)을 하나의 응답으로 스트리밍 해준다.

In [ ]:
for event in graph.stream({"messages": [("user", user_input)]}, stream_mode="values"):
    # 이벤트 값 출력
    for key, value in event.items():
        print(f"\n=================\nSTEP: {key}\n=================\n")
        print(value[-1])